In [ ]:
import re
import statsmodels

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import datetime as dt

from factor_analyzer import FactorAnalyzer, ConfirmatoryFactorAnalyzer, ModelSpecificationParser
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore, chi2, norm, multivariate_normal, spearmanr, pearsonr, shapiro
from scipy.optimize import minimize
from collections import defaultdict

In [ ]:
# Load data, drop test entry
df = pd.read_csv("qualtrics_export.csv", on_bad_lines="skip", delimiter=";")
df = df[df["Finished"] == "TRUE"].drop(2)

In [ ]:
# Calculate average time to complete survey
times = df.iloc[2:][["StartDate", "EndDate"]]
times["total"] = pd.to_datetime(times["EndDate"]) - pd.to_datetime(times["StartDate"])
times["total"].sort_values().iloc[:-1].mean()

## Drop evaluations that were made too quickly

In [ ]:
all_times = []
qs = []

for row in df.T.iloc[17:].itertuples():
    if row[0].endswith("Page Submit"):
        
        # Questions 201-361 are the actual explanation pair questions
        qnum = int(row[0].split("_")[0][1:])
        
        if 201 <= qnum <= 362:
            times = np.array([float(i) for i in row[1:]])
            times_filtered = [i for i in times if not np.isnan(i)]
            all_times.extend(times_filtered)
            qs.extend([qnum] * len(times_filtered))

all_times = np.array(all_times)
qs = np.array(qs)
b10 = np.quantile(all_times, q=.1)
print(f"Bottom 10th percetile of time per question: {b10}s")
print(f"Number of evaluations that took longer than b10: {len(qs[all_times >= b10])}")

In [ ]:
unique, counts = np.unique(qs[all_times >= b10], return_counts=True)

plt.figure(figsize=(12, 6))  # Increase figure size
plt.bar(unique, counts, color='skyblue', edgecolor='black')  # Add color and edge for clarity
plt.xlabel('Question Number', fontsize=12)
plt.ylabel('Number of Usable Answers', fontsize=12)
plt.title('Usable Answers Per Question - 10th percentile', fontsize=14)
plt.xticks(unique[::10], rotation=45, fontsize=10)  # Show every 10th tick, rotate for readability
plt.tight_layout()  # Adjust layout to avoid clipping

plt.axhline(y=10, color='red', linestyle='-', linewidth=1, label='Threshold (y=10)')

plt.show()

# General descriptives

In [ ]:
# Education
df["Q3"].iloc[2:].value_counts().plot(kind="barh")

In [ ]:
# Gender
df["Q1"].iloc[2:].value_counts().plot(kind="barh")

In [ ]:
# Age
df["Q2"].iloc[2:].dropna().apply(int).describe()

In [ ]:
# Job type
df["Q4"].iloc[2:].str.lower().value_counts()

# Converting Likert to numbers

In [ ]:
# Just the cognitive orientation questions per participant
df_co = df[["PROLIFIC_PID", "Q1.1", "Q2 ", "Q3.1", "Q4.1", "Q5", "Q6",
            "Q7", "Q8", "Q9", "Q10", "Q11", "Q12", 
            "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", 
            "Q19", "Q28", "Q29", "Q30", "Q31", "Q32", 
            "Q33"]].T

# Drop columns by testers
df_co = df_co.rename(columns = {0 : "Question text"})

# Set column names to prolific IDs
new_headers = df_co.iloc[0, 1:].values  
first_column_header = "Question text"
df_co.columns = [first_column_header] + new_headers.tolist()
df_co = df_co.drop(index = "PROLIFIC_PID")

# Rescale
df_co = df_co.replace({"Strongly disagree" : 0, "Somewhat disagree" : 1,
                       "Neither agree nor disagree" : 2, "Somewhat agree" : 3,
                       "Strongly agree" : 4})

df_co.head()

In [ ]:
co_groups = {"Need for Cognition" : {
               "Q1.1" : {"Reverse" : True},
               "Q2 "  : {"Reverse" : False},
               "Q3.1" : {"Reverse" : True},
               "Q4.1" : {"Reverse" : True},
               "Q5"   : {"Reverse" : False}
               },
            "Need for closure" : {
               "Q6"  : {"Reverse" : False},
               "Q7"  : {"Reverse" : True},
               "Q8"  : {"Reverse" : True},
               "Q9"  : {"Reverse" : False}, 
               "Q10" : {"Reverse" : False},
               "Q11" : {"Reverse" : False},
               "Q12" : {"Reverse" : False}
            },
            "Susceptibility to persuasion" : {
                "Q13" : {"Reverse" : False},
                "Q14" : {"Reverse" : False},
                "Q15" : {"Reverse" : False},
                "Q16" : {"Reverse" : False}
            },
            "Skepticism" : {
                "Q17" : {"Reverse" : False},
                "Q18" : {"Reverse" : True},
                "Q19" : {"Reverse" : False},
                "Q28" : {"Reverse" : False},
                "Q29" : {"Reverse" : False}
            },
            "AI Expertise" : {
                "Q30" : {"Reverse" : False},
                "Q31" : {"Reverse" : False},
                "Q32" : {"Reverse" : False},
                "Q33" : {"Reverse" : True}
            }
        }

sub_dfs = {}

for group, questions in co_groups.items():
    temp = df_co.loc[list(questions.keys())]
    
    for question, reverse in questions.items():
        if reverse["Reverse"]:    
            temp.loc[question] = temp.loc[question].replace({4:0, 3:1, 1:3, 0:4})
            
    temp = temp.drop(columns = "Question text")
    
    sub_dfs[group] = temp
    
sub_dfs["Need for Cognition"]

In [ ]:
all_scaled = pd.concat(list(sub_dfs.values()))

# Drop incomplete responses
all_scaled = all_scaled.drop(columns=all_scaled.columns[all_scaled.isna().sum() > 0]).T

# Confirmatory Factor Analysis

In [ ]:
# The questions per factor
model_dict = {k : list(v.keys()) for k, v in co_groups.items()}

# Run CFA
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(all_scaled, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False) 
cfa.fit(all_scaled.values) 
cfa.loadings_ 

In [ ]:
# Drop scores below the threshold (.3)
scores = cfa.loadings_
scores = np.array(scores).flatten()
scores = scores[scores != 0]

co_groups_updated = defaultdict(dict)

counter = 0

to_drop = []

for group, questions in co_groups.items():
    for question in questions:
        # Q33 is 'objective' and should therefore be included regardless of self-report alignment
        if scores[counter] > .3 or question == "Q33":
            
            co_groups_updated[group][question] = co_groups[group][question]
        
        else:
            print(f"Dropping: {question}")
            to_drop.append(question)

        counter += 1
        
co_groups = co_groups_updated
all_scaled_corrected = all_scaled.drop(columns=to_drop)

### Rerun CFA with updated questions list
# The questions per factor
model_dict = {k : list(v.keys()) for k, v in co_groups_updated.items()}

# Run CFA
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(all_scaled_corrected,
                                                                          model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False) 
cfa.fit(all_scaled_corrected.values) 
cfa.loadings_

# Calculating concept scores per participant

In [ ]:
# Calculate average scores for each concept
concept_scores = pd.DataFrame()

for concept, questions in co_groups.items():
    concept_scores[concept] = all_scaled[list(questions.keys())].sum(axis=1) / (len(questions.keys()) * 4)

# Output results
concept_scores.describe()

In [ ]:
for col in concept_scores.columns:
    print(f"{col}: {shapiro(concept_scores[col])}")

In [ ]:
# Function to plot the distributions
def plot_concept_score_distributions(dataframe):
    num_columns = len(dataframe.columns)
    num_rows = (num_columns + 2) // 3  # Arrange in rows of 3 plots

    fig, axes = plt.subplots(num_rows, 3, figsize=(15, num_rows * 5))
    axes = axes.flatten()

    for i, column in enumerate(dataframe.columns):
        sns.histplot(dataframe[column], kde=True, ax=axes[i], bins=15)
        axes[i].set_title(column)
        axes[i].set_xlabel("Score")
        axes[i].set_xlim(0, 1)
        axes[i].set_ylabel("Frequency")

    # Turn off unused axes
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()

# Example usage
plot_concept_score_distributions(concept_scores)


In [ ]:
concept_scores.to_csv("../results/cognitive_orientations.csv")

# Data restructuring (evaluations)

In [ ]:
# Check for columns related to explanation pair evaluations and participant-level identifiers
pair_columns = [col for col in df.columns if ("v" in str(col)) or ("Page Submit" in str(col))]
participant_columns = ['ResponseId', 'PROLIFIC_PID']
subset_data = df[participant_columns + pair_columns]

# Find submit times of all questions
submits = sorted([i for i in pair_columns if re.match("Q\\d{3}_Page Submit", i)])[:-4]
qs = sorted(list(set([i[:-2] for i in pair_columns if re.match("\\d{6}v\\d{6}_\\d", i)])))

lookup = dict(zip(submits, qs))

# Actually remove any evaluations that took less than b10 from the dataset
for row in subset_data.iterrows():

    index, row = row
    
    for submit in submits:
        if float(row[submit]) < b10:
            subset_data.loc[index, f"{lookup[submit]}_1"] = np.nan
            subset_data.loc[index, f"{lookup[submit]}_2"] = np.nan
            subset_data.loc[index, f"{lookup[submit]}_3"] = np.nan

subset_data

In [ ]:
# Reshape the explanation pair data into long format

# Melt explanation pair columns to create a long-format dataframe

long_data = pd.melt(
    subset_data,
    id_vars=participant_columns,  # Keep participant identifiers
    value_vars=pair_columns,  # Explanation pair columns
    var_name="ExplanationPair_Metric",  # New column for explanation pair + metric
    value_name="Evaluation"  # Column for evaluation (trust, transparency, usefulness)
)

# Split ExplanationPair_Metric into ExplanationPair and MetricType
long_data[['ExplanationPair', 'MetricType']] = long_data['ExplanationPair_Metric'].str.extract(r'(.*)_([1-3])')

# Map MetricType to human-readable labels
metric_map = {
    "1": "Trustworthiness",
    "2": "Transparency",
    "3": "Usefulness"
}

long_data['MetricType'] = long_data['MetricType'].map(metric_map)

# Drop unnecessary columns
long_data = long_data.drop(columns = ["ExplanationPair_Metric"])
long_data = long_data.dropna(subset = ["Evaluation", "ExplanationPair"])

# Display a sample of the reshaped data
long_data.sort_values(by=["PROLIFIC_PID", "ExplanationPair"]).head()

In [ ]:
def flip_values(pair):
    """
    The bits used to represent the explanations are counterintuitive for
    Persuasion, Detail, and Formality. Hence, we flip them. As a result, the coding is:
    bit 0: 0 = low-stakes,       1 = high-stakes
    bit 1: 0 = short,            1 = long
    bit 2: 0 = running,          1 = bulleted
    bit 3: 0 = informal,         1 = formal
    bit 4: 0 = aggregated,       1 = comprehensive
    bit 5: 0 = decision-support, 1 = persuasive
    """
    
    pair = list(pair)
    pair[3], pair[10] = "0" if int(pair[3]) else "1", "0" if int(pair[10]) else "1"
    pair[4], pair[11] = "0" if int(pair[4]) else "1", "0" if int(pair[11]) else "1"
    pair[5], pair[12] = "0" if int(pair[5]) else "1", "0" if int(pair[12]) else "1"
 
    
    return "".join(pair)
    
long_data["ExplanationPair"] = long_data["ExplanationPair"].apply(flip_values)

In [ ]:
def pair_parser(explanation_pair):
    """
    Converts the string-like representation of the explanation pair
    to a categorical variable indicating which of the design dimensions
    has been 'flipped' between the two explanations. In this flip, explanation
    B always has a 1 ('higher' value) and explanation A a 0 ('lower' value)
    """
    
    explanation_a, explanation_b = explanation_pair.split("v")
    feature_names = ["Domain", "Length", "Structure", "Formality", "Detail", "Persuasiveness"]
    
    features_a = {f"{feature}": int(bit) for feature, bit in zip(feature_names, explanation_a)}
    features_b = {f"{feature}": int(bit) for feature, bit in zip(feature_names, explanation_b)}       

    return {"Domain": "high" if features_a["Domain"] else "low", 
            "flipped_dim": feature_names[np.argmax([features_a[feature] != features_b[feature] 
                                         for feature in feature_names])]}
        
# Apply the function to the ExplanationPair column and expand the result into new columns
binary_features = long_data['ExplanationPair'].map(pair_parser).apply(pd.Series)

# Concatenate the binary features back into the main dataframe
long_data = pd.concat([long_data, binary_features], axis=1)

# Drop any unnecessary intermediate columns
long_data = long_data.drop(columns=["ResponseId", "Features_A", "Features_B"],
                           errors="ignore")

In [ ]:
# Include flipped_dim and domain as a column
cols = ["PROLIFIC_PID", "ExplanationPair", "Need for Cognition",
        "Need for closure", "Susceptibility to persuasion",
        "Skepticism", "AI Expertise", "Domain", "flipped_dim", 
        "MetricType", "Evaluation"]
    
# Merge evaluations with cognitive orientations and restructure
final_data = long_data.join(concept_scores, 
                            on="PROLIFIC_PID", 
                            how="left").sort_values(
    by=["PROLIFIC_PID", "ExplanationPair"]
)[cols]

final_data["Evaluation"] = final_data["Evaluation"].replace({"Explanation A": 0, "Explanation B": 1})

final_data.head()

In [ ]:
def decision_flipper(row):
    """
    Since we flipped three dimensions, their direction in terms of evaluation
    has also been flipped (i.e., explanation A (baseline) has "become" explanation B (treatment)
    since that one now has the 1 instead of the 0. As a result, we also have to flip the order of the 
    explanations and the evaluation for it, so that a 1 again indicates a preference for the treatment. 
    """

    split = row["ExplanationPair"].split("v")
    reversed_order = f"{split[1]}v{split[0]}"
    
    if row["flipped_dim"] in ["Formality", "Detail", "Persuasiveness"]:
        return 0 if row["Evaluation"] else 1, reversed_order 
    else:
        return row["Evaluation"], row["ExplanationPair"]

# Apply flipper
for row in final_data.iterrows():
    fixed_eval, fixed_explanation = decision_flipper(row[1])
    
    final_data.loc[row[0], "Evaluation"] = fixed_eval
    final_data.loc[row[0], "ExplanationPair"] = fixed_explanation

final_data.head()

# Calculate polychoric correlation

In [ ]:
# Store correlation dataset as csv
correlation_df = final_data[["PROLIFIC_PID", 
                             "MetricType", 
                             "Evaluation",
                             "ExplanationPair"]].pivot(
                     columns="MetricType", 
                     values="Evaluation",
                     index=["PROLIFIC_PID", "ExplanationPair"]
                 )

correlation_df.to_csv("../results/correlation_df.csv")

In [ ]:
%%R
library(polycor)

data <- read.csv("correlation_df.csv")

calculate_p <- function(corr) {
  rho = corr$rho
  SE = sqrt(corr$var)
  z <- rho / SE
  p = 2 * (1 - pnorm(abs(z)))
  return(as.numeric(p))
}

corr_use_trans = polychor(data$Usefulness, data$Transparency, std.err=TRUE)
p_use_trans = calculate_p(corr_use_trans)

corr_use_trust = polychor(data$Usefulness, data$Trustworthiness, std.err=TRUE)
p_use_trust = calculate_p(corr_use_trust)

corr_trust_trans = polychor(data$Trustworthiness, data$Transparency, std.err=TRUE)
p_trust_trans = calculate_p(corr_trust_trans)

print(corr_use_trans$rho)
print(p_use_trans)
print(corr_use_trust$rho)
print(p_use_trust)
print(corr_trust_trans$rho)
print(p_trust_trans)

In [ ]:
final_data = final_data.reset_index().drop(columns="index")
final_data.columns = [i.replace(" ", "_") for i in final_data.columns]
final_data = final_data.dropna()

# Calculate share of evaluations where all metrics aligned
decisions = final_data.groupby(["PROLIFIC_PID", "ExplanationPair"])["Evaluation"].sum().values
np.isin(decisions, [0, 3]).mean()

# Logistic Regression

In [ ]:
# Store dataframe used for logistic regressions
final_data.to_csv("../results/evaluations.csv")

final_data.head()

## Fitting the mixed-effects logistic regression model (without personalization)

In [ ]:
%%R

# Load required libraries
library(lme4)
library(dplyr)
library(stats)

# Load the data
data <- read.csv("../results/evaluations.csv")

# Set categorical variables
data$flipped_dim <- factor(data$flipped_dim, 
                           levels = c("Structure", "Length", "Formality", "Detail", "Persuasiveness"))

data$MetricType <- factor(data$MetricType, 
                          levels = c("Trustworthiness", "Usefulness", "Transparency"))

data$Domain <- factor(data$Domain, 
                      levels = c("high", "low"))

# Fit the mixed-effects logistic regression model
model <- glmer(Evaluation ~ flipped_dim + MetricType + Domain +
                 (1 | PROLIFIC_PID), 
               data = data, 
               family = binomial,
               control = glmerControl(optimizer = "bobyqa", 
                                      optCtrl = list(maxfun = 1e6))
               )

print(summary(model))

# Extract fixed effect coefficients
coefs <- fixef(model)

# Extract variance-covariance matrix
vcov_matrix <- vcov(model)

# Separate main effects and interaction terms
main_effects <- coefs[grepl("^flipped_dim", names(coefs))]

# Derive the missing coefficient for 'Structure' (main effect)
structure_main_effect <- -sum(main_effects)

# Calculate standard errors for the missing coefficients
# Main effect for 'Structure'
main_effect_vars <- diag(vcov_matrix)[grepl("^flipped_dim", names(coefs))]
main_effect_covs <- vcov_matrix[grepl("^flipped_dim", names(coefs)), grepl("^flipped_dim", names(coefs))]
main_effect_se <- sqrt(sum(main_effect_vars) + 2 * sum(main_effect_covs[lower.tri(main_effect_covs)]))

# Calculate z-scores and p-values
structure_main_z <- structure_main_effect / main_effect_se
structure_main_p <- 2 * (1 - pnorm(abs(structure_main_z)))

# Combine results into a data frame
# Main effects
main_results <- data.frame(
  Predictor = gsub("flipped_dim", "", names(main_effects)),
  Coefficient = main_effects,
  SE = sqrt(diag(vcov_matrix)[grepl("^flipped_dim", names(coefs))]),
  z = main_effects / sqrt(diag(vcov_matrix)[grepl("^flipped_dim", names(coefs))]),
  p_value = 2 * (1 - pnorm(abs(main_effects / sqrt(diag(vcov_matrix)[grepl("^flipped_dim", names(coefs))]))))
)

# Missing coefficients (Structure)
structure_results <- data.frame(
  Predictor = "Structure",
  Coefficient = structure_main_effect,
  SE = main_effect_se,
  z = structure_main_z,
  p_value = structure_main_p
)

# Combine all results, explicitly including the intercept
intercept_result <- data.frame(
  Predictor = "(Intercept)",
  Coefficient = coefs["(Intercept)"],
  SE = sqrt(diag(vcov_matrix)["(Intercept)"]),
  z = coefs["(Intercept)"] / sqrt(diag(vcov_matrix)["(Intercept)"]),
  p_value = 2 * (1 - pnorm(abs(coefs["(Intercept)"] / sqrt(diag(vcov_matrix)["(Intercept)"]))))
)

# Combine all results
final_results <- bind_rows(intercept_result, main_results, structure_results)

# Format the table and include significance levels
final_results <- final_results %>%
  mutate(
    Coefficient = round(Coefficient, 3),
    SE = round(SE, 3),
    z = round(z, 3),
    p_value = round(p_value, 3),
    Significance = case_when(
      p_value < 0.001 ~ "***",
      p_value < 0.01 ~ "**",
      p_value < 0.05 ~ "*",
      TRUE ~ ""
    )
  )

# Fix ordering
rownames(final_results) <- NULL
custom_order <- c("(Intercept)", "Detail", "Formality", "Length", "Persuasiveness", "Structure")

final_results <- final_results[match(custom_order, final_results$Predictor), ]

write.csv(final_results, "../results/final_results_general.csv", row.names = FALSE)

In [ ]:
Routput = pd.read_csv("../results/final_results_general.csv")

corr = statsmodels.stats.multitest.fdrcorrection(Routput["p_value"],
                                                 alpha=0.05, 
                                                 method='indep',
                                                 is_sorted=False)

Routput["Odds Ratio"] = np.exp(Routput["Coefficient"])
Routput["Corrected p-value"] = corr[1]
Routput["Reject H0"] = corr[0]

Routput[["Predictor", "Coefficient", "SE", "Odds Ratio", "Corrected p-value"]]

## Fitting the mixed-effects logistic regression model (with personalization)

In [ ]:
%%R

# Load required libraries
library(lme4)
library(dplyr)
library(stats)

# Load the data
data <- read.csv("../results/evaluations.csv")

# Set categorical variables
data$flipped_dim <- factor(data$flipped_dim, 
                           levels = c("Structure", "Length", "Formality", "Detail", "Persuasiveness"))

data$MetricType <- factor(data$MetricType, 
                          levels = c("Trustworthiness", "Usefulness", "Transparency"))

data$Domain <- factor(data$Domain, 
                      levels = c("high", "low"))

# Fit the mixed-effects logistic regression model
model <- glmer(Evaluation ~ Need_for_Cognition + Need_for_closure + 
                 Susceptibility_to_persuasion + Skepticism + 
                 AI_Expertise + flipped_dim + MetricType + Domain +
                 (Need_for_Cognition + Need_for_closure + Susceptibility_to_persuasion + 
                    Skepticism + AI_Expertise):flipped_dim + 
                 (1 | PROLIFIC_PID), 
               data = data, 
               family = binomial,
               control = glmerControl(optimizer = "bobyqa", 
                                      optCtrl = list(maxfun = 1e6))
               )

print(summary(model))

# Extract fixed effect coefficients
coefs <- fixef(model)

# Extract variance-covariance matrix
vcov_matrix <- vcov(model)

# Separate main effects and interaction terms
main_effects <- coefs[grepl("^flipped_dim", names(coefs))]
interaction_terms <- coefs[grepl(":flipped_dim", names(coefs))]
continuous_effects <- coefs[!grepl("flipped_dim", names(coefs)) & !grepl(":", names(coefs))]

# Derive the missing coefficient for 'Structure' (main effect)
structure_main_effect <- -sum(main_effects)

# Derive the missing coefficients for 'Structure' (interactions)
predictors <- c("Need_for_Cognition", "Need_for_closure", 
                "Susceptibility_to_persuasion", "Skepticism", "AI_Expertise")
structure_interactions <- sapply(predictors, function(pred) {
  interaction_coefs <- coefs[grepl(paste0(pred, ":flipped_dim"), names(coefs))]
  -sum(interaction_coefs)
})

# Calculate standard errors for the missing coefficients
# Main effect for 'Structure'
main_effect_vars <- diag(vcov_matrix)[grepl("^flipped_dim", names(coefs))]
main_effect_covs <- vcov_matrix[grepl("^flipped_dim", names(coefs)), grepl("^flipped_dim", names(coefs))]
main_effect_se <- sqrt(sum(main_effect_vars) + 2 * sum(main_effect_covs[lower.tri(main_effect_covs)]))

# Interactions for 'Structure'
interaction_se <- sapply(1:length(predictors), function(i) {
  pred <- predictors[i]
  interaction_indices <- which(grepl(paste0(pred, ":flipped_dim"), names(coefs)))
  interaction_vars <- diag(vcov_matrix)[interaction_indices]
  interaction_covs <- vcov_matrix[interaction_indices, interaction_indices]
  sqrt(sum(interaction_vars) + 2 * sum(interaction_covs[lower.tri(interaction_covs)]))
})

# Calculate z-scores and p-values
structure_main_z <- structure_main_effect / main_effect_se
structure_main_p <- 2 * (1 - pnorm(abs(structure_main_z)))

interaction_z <- structure_interactions / interaction_se
interaction_p <- 2 * (1 - pnorm(abs(interaction_z)))

# Combine results into a data frame
# Main effects
main_results <- data.frame(
  Predictor = gsub("flipped_dim", "", names(main_effects)),
  Coefficient = main_effects,
  SE = sqrt(diag(vcov_matrix)[grepl("^flipped_dim", names(coefs))]),
  z = main_effects / sqrt(diag(vcov_matrix)[grepl("^flipped_dim", names(coefs))]),
  p_value = 2 * (1 - pnorm(abs(main_effects / sqrt(diag(vcov_matrix)[grepl("^flipped_dim", names(coefs))]))))
)

# Continuous predictors (retain as-is)
continuous_results <- data.frame(
  Predictor = names(continuous_effects),
  Coefficient = continuous_effects,
  SE = sqrt(diag(vcov_matrix)[!grepl("flipped_dim", names(coefs)) & !grepl(":", names(coefs))]),
  z = continuous_effects / sqrt(diag(vcov_matrix)[!grepl("flipped_dim", names(coefs)) & !grepl(":", names(coefs))]),
  p_value = 2 * (1 - pnorm(abs(continuous_effects / sqrt(diag(vcov_matrix)[!grepl("flipped_dim", names(coefs)) & !grepl(":", names(coefs))]))))
)

# Interaction terms
interaction_results <- data.frame(
  Predictor = gsub(":flipped_dim", ":", names(interaction_terms)),
  Coefficient = interaction_terms,
  SE = sqrt(diag(vcov_matrix)[grepl(":flipped_dim", names(coefs))]),
  z = interaction_terms / sqrt(diag(vcov_matrix)[grepl(":flipped_dim", names(coefs))]),
  p_value = 2 * (1 - pnorm(abs(interaction_terms / sqrt(diag(vcov_matrix)[grepl(":flipped_dim", names(coefs))]))))
)

# Missing coefficients (Structure)
structure_results <- data.frame(
  Predictor = c("Structure", paste0(predictors, ":Structure")),
  Coefficient = c(structure_main_effect, structure_interactions),
  SE = c(main_effect_se, interaction_se),
  z = c(structure_main_z, interaction_z),
  p_value = c(structure_main_p, interaction_p)
)

# Combine all results
final_results <- bind_rows(continuous_results, main_results, interaction_results, structure_results)

# Format the table and include significance levels
final_results <- final_results %>%
  mutate(
    Coefficient = round(Coefficient, 3),
    SE = round(SE, 3),
    z = round(z, 3),
    p_value = round(p_value, 3),
    Significance = case_when(
      p_value < 0.001 ~ "***",
      p_value < 0.01 ~ "**",
      p_value < 0.05 ~ "*",
      TRUE ~ ""
    )
  )

# Fix ordering
rownames(final_results) <- NULL
custom_order <- c("(Intercept)", "Need_for_Cognition", "Need_for_closure", "Susceptibility_to_persuasion",
                  "Skepticism", "AI_Expertise", "Detail", "Formality", "Length", "Persuasiveness", "Structure",
                  "Need_for_Cognition:Detail", "Need_for_Cognition:Formality", "Need_for_Cognition:Length",
                  "Need_for_Cognition:Persuasiveness", "Need_for_Cognition:Structure", "Need_for_closure:Detail", 
                  "Need_for_closure:Formality", "Need_for_closure:Length", "Need_for_closure:Persuasiveness", 
                  "Need_for_closure:Structure", "Susceptibility_to_persuasion:Detail", "Susceptibility_to_persuasion:Formality",
                  "Susceptibility_to_persuasion:Length", "Susceptibility_to_persuasion:Persuasiveness",
                  "Susceptibility_to_persuasion:Structure", "Skepticism:Detail", "Skepticism:Formality", 
                  "Skepticism:Length", "Skepticism:Persuasiveness", "Skepticism:Structure", "AI_Expertise:Detail",
                  "AI_Expertise:Formality","AI_Expertise:Length","AI_Expertise:Persuasiveness","AI_Expertise:Structure")

final_results <- final_results[match(custom_order, final_results$Predictor), ]

write.csv(final_results, "../results/final_results_personal.csv", row.names = FALSE)

In [ ]:
Routput = pd.read_csv("../results/final_results_personal.csv")

corr = statsmodels.stats.multitest.fdrcorrection(Routput["p_value"],
                                                 alpha=0.05, 
                                                 method='indep',
                                                 is_sorted=False)

Routput["Odds Ratio"] = np.exp(Routput["Coefficient"])
Routput["Corrected p-value"] = corr[1]
Routput["Reject H0"] = corr[0]

Routput[["Predictor", "Coefficient", "SE", "Odds Ratio", "z", "Corrected p-value", "Reject H0"]]